In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('basic').getOrCreate()

24/11/23 00:35:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/23 00:35:12 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/11/23 00:35:12 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/11/23 00:35:12 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
24/11/23 00:35:12 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [2]:
from pyspark.sql.functions import col,current_timestamp,count,countDistinct,sum,aggregate,desc,rank,when
from pyspark.sql.window import Window

In [3]:
race_result_agg_df = spark.read.option('header',True).option('inferSchema',True).format('csv').load('../tbls/presentation/race_results_twoyears/race_results_twoyears.csv')\
    .filter('race_year=2020')


In [4]:
driver_standing_df = race_result_agg_df.groupBy('race_year','driver_name','driver_nationality','team')\
    .agg(sum('points').alias('total_points'),\
    count(when(col('position') ==1,True)).alias('wins'))

In [12]:
window_spec = Window.partitionBy('race_year')\
    .orderBy(desc('total_points'),desc('wins'))
driverstanding = driver_standing_df.withColumn('rank',rank().over(window_spec))

In [13]:
driverstanding.write.option('header',True).mode('overwrite').format('csv').save('../tbls/presentation/driverstanding/driverstanding.csv')
                        